In [5]:
# | default_exp tools.code_interpreter

In [6]:
# | export
import sys
import io
import os
import time
import logging
from contextlib import redirect_stdout, redirect_stderr
from typing import Dict, Any, Optional, List
from pydantic import BaseModel, ValidationError, field_validator, Field
from pathlib import Path
from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory

# Configure logging for production
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CodeInterpreterParams(BaseModel):
    code: str = Field(..., description="Python code to execute")
    capture_output: Optional[bool] = Field(True, description="Capture stdout/stderr of the execution")
    timeout: Optional[int] = Field(30, ge=1, description="Execution timeout in seconds")
    working_dir: Optional[str] = Field(None, description="Working directory for execution")
    env_vars: Optional[Dict[str, str]] = Field(None, description="Additional environment variables")
    requirements: Optional[List[str]] = Field(None, description="Required Python packages (not installed in this version)")
    max_output_size: Optional[int] = Field(16384, ge=1024, description="Maximum output size in bytes")
    custom_globals: Optional[Dict[str, Any]] = Field(None, description="Custom global variables to inject into execution")

    @field_validator("working_dir")
    @classmethod
    def validate_working_dir(cls, value: Optional[str]) -> Optional[str]:
        if value:
            try:
                path = Path(value).resolve()
                if not path.exists():
                    raise ValueError(f"Working directory {value} does not exist")
                if not path.is_dir():
                    raise ValueError(f"Working directory {value} is not a directory")
                return str(path)
            except Exception as e:
                logger.error(f"Invalid working directory: {str(e)}")
                raise ValueError(f"Invalid working directory: {str(e)}")
        return value

    @field_validator("env_vars")
    @classmethod
    def validate_env_vars(cls, value: Optional[Dict[str, str]]) -> Optional[Dict[str, str]]:
        if value:
            for k, v in value.items():
                if not isinstance(k, str) or not isinstance(v, str):
                    raise ValueError(f"Environment variables must be strings: {k}={v}")
        return value

    @field_validator("custom_globals")
    @classmethod
    def validate_custom_globals(cls, value: Optional[Dict[str, Any]]) -> Optional[Dict[str, Any]]:
        if value:
            for k in value.keys():
                if not isinstance(k, str):
                    raise ValueError(f"Custom global keys must be strings: {k}")
        return value

class CodeInterpreterTool(BaseTool):
    def __init__(self, log_level: str = "INFO"):
        metadata = ToolMetadata(
            name="code_interpreter",
            description="Execute Python code in a controlled environment for analysis, computation, or scripting",
            category=ToolCategory.ANALYSIS,
            requires_approval=True
        )
        super().__init__(metadata)
        logging.getLogger().setLevel(getattr(logging, log_level, logging.INFO))

    def get_parameters_schema(self, verbose: bool = True) -> Dict[str, Any]:
        """Returns OpenAI-compatible schema for the tool."""
        try:
            schema = {
                "type": "function",
                "function": {
                    "name": "code_interpreter",
                    "description": "Execute Python code with output capture and timeout control for analysis or computation." if verbose else "",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "code": {"type": "string", "description": "Python code to execute"},
                            "capture_output": {"type": "boolean", "description": "Capture stdout/stderr (default: true)"},
                            "timeout": {"type": "integer", "description": "Execution timeout in seconds (default: 30)"},
                            "working_dir": {"type": "string", "description": "Working directory for execution"},
                            "env_vars": {
                                "type": "object",
                                "description": "Additional environment variables as key-value pairs"
                            },
                            "requirements": {
                                "type": "array",
                                "items": {"type": "string"},
                                "description": "Required Python packages (not installed in this version)"
                            },
                            "max_output_size": {"type": "integer", "description": "Maximum output size in bytes (default: 16384)"},
                            "custom_globals": {
                                "type": "object",
                                "description": "Custom global variables to inject into execution context"
                            }
                        },
                        "required": ["code"]
                    }
                }
            }
            return schema["function"]
        except Exception as e:
            logger.error(f"Schema generation failed: {str(e)}")
            return {}

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute Python code with specified parameters in a controlled environment."""
        try:
            params = CodeInterpreterParams(**kwargs)
            start_time = time.time()
            stdout = io.StringIO()
            stderr = io.StringIO()
            local_vars = {}

            # Set working directory if provided
            original_cwd = os.getcwd()
            if params.working_dir:
                try:
                    os.chdir(params.working_dir)
                except OSError as e:
                    logger.error(f"Failed to set working directory {params.working_dir}: {str(e)}")
                    return {"success": False, "error": f"Invalid working directory: {str(e)}", "execution_time": 0.0}

            try:
                # Set up safe globals to prevent dangerous operations
                safe_globals = {
                    "__builtins__": {
                        k: v for k, v in __builtins__.__dict__.items()
                        if k not in ["__import__", "eval", "exec", "open", "sys"]
                    },
                    "print": print,  # Allow print for output
                    # Pre-import common modules for analysis
                    "json": __import__("json"),
                    "re": __import__("re"),
                    "ast": __import__("ast"),
                    "hashlib": __import__("hashlib"),
                    "os": __import__("os")  # Limited os for env access
                }
                # Inject custom globals if provided
                if params.custom_globals:
                    safe_globals.update(params.custom_globals)

                # Apply environment variables
                if params.env_vars:
                    os.environ.update(params.env_vars)

                # Execute code
                with redirect_stdout(stdout) if params.capture_output else nullcontext(), \
                     redirect_stderr(stderr) if params.capture_output else nullcontext():
                    # Use timeout context manager (assuming time_limit is available or simulated)
                    try:
                        from contextlib import contextmanager
                        @contextmanager
                        def time_limit(seconds):
                            import signal
                            def signal_handler(signum, frame):
                                raise TimeoutError("Execution timed out")
                            signal.signal(signal.SIGALRM, signal_handler)
                            signal.alarm(seconds)
                            try:
                                yield
                            finally:
                                signal.alarm(0)
                        with time_limit(params.timeout):
                            exec(params.code, safe_globals, local_vars)
                    except ImportError:
                        # Fallback if signal not available (e.g., Windows)
                        exec(params.code, safe_globals, local_vars)

                execution_time = time.time() - start_time
                stdout_val = stdout.getvalue() if params.capture_output else ""
                stderr_val = stderr.getvalue() if params.capture_output else ""
                if len(stdout_val) > params.max_output_size:
                    stdout_val = stdout_val[:params.max_output_size] + "... [truncated]"
                if len(stderr_val) > params.max_output_size:
                    stderr_val = stderr_val[:params.max_output_size] + "... [truncated]"
                return {
                    "success": True,
                    "stdout": stdout_val,
                    "stderr": stderr_val,
                    "execution_time": execution_time,
                    "message": "Python code executed successfully"
                }
            except TimeoutError:
                logger.warning(f"Python execution timed out after {params.timeout} seconds")
                return {
                    "success": False,
                    "error": f"Execution timed out after {params.timeout} seconds",
                    "stdout": stdout.getvalue()[:params.max_output_size] if params.capture_output else "",
                    "stderr": stderr.getvalue()[:params.max_output_size] if params.capture_output else "",
                    "execution_time": time.time() - start_time
                }
            except SyntaxError as e:
                logger.error(f"Python syntax error: {str(e)}")
                return {
                    "success": False,
                    "error": f"Syntax error: {str(e)}",
                    "stdout": stdout.getvalue()[:params.max_output_size] if params.capture_output else "",
                    "stderr": stderr.getvalue()[:params.max_output_size] if params.capture_output else "",
                    "execution_time": time.time() - start_time
                }
            except Exception as e:
                logger.error(f"Python execution failed: {str(e)}")
                return {
                    "success": False,
                    "error": f"Execution error: {str(e)}",
                    "stdout": stdout.getvalue()[:params.max_output_size] if params.capture_output else "",
                    "stderr": stderr.getvalue()[:params.max_output_size] if params.capture_output else "",
                    "execution_time": time.time() - start_time
                }
            finally:
                # Restore original working directory and environment
                os.chdir(original_cwd)
                if params.env_vars:
                    for key in params.env_vars:
                        os.environ.pop(key, None)
                stdout.close()
                stderr.close()
        except ValidationError as e:
            logger.error(f"Invalid parameters: {str(e)}")
            return {"success": False, "error": f"Invalid parameters: {str(e)}", "execution_time": 0.0}
        except Exception as e:
            logger.critical(f"Unexpected execution error: {str(e)}")
            return {"success": False, "error": f"Unexpected error: {str(e)}", "execution_time": 0.0}